## Import Necessary Libraries

In [1]:
import GPy
import numpy as np
import pandas as pd
import time
# Set Seed
np.random.seed(0)

## Read Train And Test Files

#### Make Sure you keep all these files in the current directory

In [2]:
# Read Files
X_train = np.load('X_train.npy') 
X_test  = np.load('X_test.npy')
Y_train = np.load('Y_train.npy')
Y_test  = np.load('Y_test.npy') 

# Make Y_train and Y_test as 2d array for model functions fitting
Y_train = Y_train[:,None]
Y_test = Y_test[:,None]

## Model Building and Prediction

In [3]:
start_time = time.time() # To save start time of execution of code

try:
 
    # Initialize Dimensions
    Dimensions = X_train.shape[1]

    # Kernel Building
    k_rbf = GPy.kern.RBF(input_dim=Dimensions, variance=1, lengthscale=None, ARD=True
                         , active_dims=None, name='rbf', useGPU=False, inv_l=False)

    # Model Building
    model_rbf = GPy.models.GPClassification(X_train, Y_train,kernel=k_rbf)  
    
    # Optimize the model
    model_rbf.optimize(optimizer=None,start=None,messages=False, max_iters=1000
                       , ipython_notebook=False, clear_after_finish=True) 

    # Prediction
    prediction_proba= model_rbf.predict(X_test)[0] 

    # Save Predicted Probability and Actual label in variable
    True_label = Y_test.flatten()
    Predicted_probability = prediction_proba.flatten().round(2)

except:
    print('Jitter Error')
    # If jitter error occurs try to reset np.random.seed() with some other value
    
end_time = time.time()    
print("Execution Time is ", end_time-start_time, "seconds")

Execution Time is  3.4108762741088867 seconds


### Save Prediction Result into One DataFrame

In [4]:
Classification_result = pd.DataFrame(columns = ['True_Label','Predicted_Probability','Predicted_Label'])
Classification_result['True_Label'] = True_label
Classification_result['Predicted_Probability'] = Predicted_probability
Classification_result['Predicted_Label']  = -1 # Initialize column with -1
# Since predicted probability >=0.5 indicate prediction of label 1 we create one more column to save predicted labels
idx_where_predicted_one = np.where(Classification_result['Predicted_Probability']>=0.5)[0]
Classification_result.loc[idx_where_predicted_one,'Predicted_Label'] = 1

In [5]:
Classification_result

,True_Label,Predicted_Probability,Predicted_Label
0,1,0.95,1
1,1,0.83,1
2,1,0.77,1
3,1,0.96,1
4,-1,0.18,-1
5,-1,0.39,-1
6,-1,0.18,-1
7,1,0.20,-1
8,-1,0.18,-1
9,1,0.39,-1


## Model Evaluation

In [6]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

In [7]:
conf_mat = confusion_matrix(Classification_result['True_Label'],Classification_result['Predicted_Label'],labels=[1,-1])
conf_mat

array([[14,  6],
       [ 1, 19]], dtype=int64)

In [8]:
print('Accuracy -- > ',accuracy_score(Classification_result['True_Label'],Classification_result['Predicted_Label']))
print('Precision -- > ',precision_score(Classification_result['True_Label'],Classification_result['Predicted_Label']))
print('Recall -- > ',recall_score(Classification_result['True_Label'],Classification_result['Predicted_Label']))
print('F1-Score -- > ',f1_score(Classification_result['True_Label'],Classification_result['Predicted_Label']))

Accuracy -- >  0.825
Precision -- >  0.9333333333333333
Recall -- >  0.7
F1-Score -- >  0.8
